In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
import requests
from bs4 import BeautifulSoup

In [6]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [7]:
url = 'https://finance.naver.com/item/sise_day.naver?code=005930&page=11'

In [8]:
res = requests.get(url)
soup = BeautifulSoup(res.text, 'html.parser')

In [9]:
pd.read_html(res.text)

ValueError: No tables found

In [14]:
driver = webdriver.Chrome()
driver.get(url)
html = driver.page_source

In [32]:
df = pd.read_html(html)
df = df[0]
df.dropna(inplace=True)
df

,날짜,종가,전일비,시가,고가,저가,거래량
1,2021.09.09,75300.0,1000.0,76400.0,76600.0,75000.0,17600770.0
2,2021.09.08,76300.0,200.0,76000.0,76400.0,75600.0,11798147.0
3,2021.09.07,76100.0,1200.0,77100.0,77100.0,75900.0,13239401.0
4,2021.09.06,77300.0,700.0,76800.0,77600.0,76600.0,12861180.0
5,2021.09.03,76600.0,600.0,76400.0,76700.0,76000.0,12096419.0
9,2021.09.02,76000.0,800.0,76800.0,76800.0,75700.0,15347486.0
10,2021.09.01,76800.0,100.0,76700.0,77100.0,75900.0,16114775.0
11,2021.08.31,76700.0,2100.0,74900.0,76700.0,74300.0,24630370.0
12,2021.08.30,74600.0,300.0,75400.0,75500.0,74200.0,12686999.0
13,2021.08.27,74300.0,300.0,74300.0,75000.0,73800.0,15172748.0


In [18]:
df = pd.read_csv('./data/universe_code.csv')
codes = df.종목코드.apply(lambda x: x[1:]).values
codes

array(['000210', '000220', '000240', '000250', '000660', '000990',
       '001340', '001390', '001500', '001750', '001800', '001820',
       '001880', '002310', '002350', '002390', '002870', '003090',
       '003220', '003410', '003530', '003550', '003670', '003780',
       '003850', '004000', '004170', '004490', '004590', '005290',
       '005670', '005690', '005880', '005930', '006060', '006220',
       '006390', '006400', '006620', '006650', '006740', '007330',
       '007340', '007570', '007700', '008490', '009150', '009420',
       '009780', '009970', '010050', '010130', '010240', '010640',
       '010780', '011040', '011170', '011330', '011780', '012330',
       '012510', '012630', '012700', '012750', '012790', '013030',
       '013120', '013810', '013890', '014570', '014680', '014830',
       '015230', '015710', '016090', '016360', '016580', '016600',
       '016610', '017040', '017510', '017890', '017960', '018250',
       '018260', '018310', '019550', '020000', '020150', '0207

In [20]:
import MySQLdb
db = MySQLdb.connect(host='localhost', user='root', password='1234')
cursor = db.cursor()

In [24]:
sql ='show databases;'

cursor.execute(sql)
cursor.fetchall()

(('information_schema',),
 ('budongsan',),
 ('employees',),
 ('fin_db',),
 ('gmarketdb',),
 ('mysql',),
 ('performance_schema',),
 ('sakila',),
 ('sys',),
 ('tabledb',),
 ('world',))

In [25]:
sql = 'use fin_db'
cursor.execute(sql)

0

In [29]:
sql = """
create table if not exists stock_table(
code char(6),
date date,
close float,
margin float,
start float,
high float,
low float,
volume float
)
"""

cursor.execute(sql)

0

In [34]:
import time

In [42]:
url = 'https://finance.naver.com/item/sise_day.naver?code={}&page={}'

for code in codes[:3]:
    print(code, "=============")
    for i in range(1, 3):
        driver.get(url.format(code, i))
        time.sleep(0.5)
        html = driver.page_source
        
        df = pd.read_html(html)[0]
        df.dropna(inplace=True)
        # print(df)
        print('------------')
        
        for i in range(len(df)):
            code = code
            date = df.iloc[i][0]
            close = df.iloc[i][1]
            margin = df.iloc[i][2]
            start = df.iloc[i][3]
            high = df.iloc[i][4]
            row = df.iloc[i][5]
            volumn = df.iloc[i][6]
            
            sql = """
            insert into stock_table values('{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}')
            """.format(code, date, close, margin, start, high, row, volumn)
            
            cursor.execute(sql)
driver.close()

000210 =============
------------
------------
000220 =============
------------
------------
000240 =============
------------
------------


In [43]:
sql = "select * from stock_table;"

st_price = pd.read_sql(sql, db)
st_price

,code,date,close,margin,start,high,low,volume
0,000210,2022-02-09,60200.0,600.0,59700.0,60400.0,59000.0,11911.0
1,000210,2022-02-08,59600.0,1000.0,59100.0,61000.0,58600.0,83130.0
2,000210,2022-02-07,58600.0,1100.0,57500.0,58800.0,57000.0,53386.0
3,000210,2022-02-04,57500.0,900.0,56900.0,57800.0,56600.0,34228.0
4,000210,2022-02-03,56600.0,700.0,56300.0,57500.0,56300.0,44961.0
5,000210,2022-01-28,55900.0,1700.0,54400.0,57500.0,53900.0,95644.0
6,000210,2022-01-27,54200.0,2800.0,57000.0,57100.0,53900.0,97345.0
7,000210,2022-01-26,57000.0,100.0,56600.0,57800.0,56600.0,39639.0
8,000210,2022-01-25,57100.0,1600.0,58200.0,58500.0,56600.0,106342.0
9,000210,2022-01-24,58700.0,1700.0,59300.0,61000.0,58300.0,120883.0


In [47]:
st_price.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   code    60 non-null     object 
 1   date    60 non-null     object 
 2   close   60 non-null     float64
 3   margin  60 non-null     float64
 4   start   60 non-null     float64
 5   high    60 non-null     float64
 6   low     60 non-null     float64
 7   volume  60 non-null     float64
dtypes: float64(6), object(2)
memory usage: 3.9+ KB


In [ ]:
import datetime